In [4]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools
import sympy as sp


In [5]:

r1, r2, r3, r4, a1, a2 = 0, 0.400, 0.455, 10**(-3)*np.sqrt(420**2+35**2), 0.025, 0.035
køddevinkel = np.arctan2(35, 240)


T_SB = np.array([[0, 0, 1, a1+r3+0.420+0.080],
                 [0, 1, 0, 0],
                 [1, 0, 0, r2+a2],
                 [0, 0, 0, 1]])


# def pointFromT(T=np.array) -> np.array:
#     P = np.zeros(3)
#     for i in range(len(T)):
#         P[i:] = T[i][3]
#     return P

# # print(pointFromT(T_SB))


# def makeT_SW(T_SB=np.array) -> np.array:
#     T_BW = np.array([[1, 0, 0, 0],
#                      [0, 1, 0, 0],
#                      [0, 0, 1, -0.080],
#                      [0, 0, 0, 1]])
#     return np.dot(T_SB, T_BW)


# def inverseKinematicsTheta123(T_SB:np.array) -> np.array:
#     r1, r2, r3, r4, a1, a2 = 0, 0.400, 0.455, 10**(-3)*np.sqrt(420**2+35**2), 0.025, 0.035
#     T_SW = tools.makeT_SW(T_SB)
#     P_W = tools.pointFromT(T_SW)

#     theta1_i = np.arctan2(-P_W[1], P_W[0])
#     theta1_ii = np.arctan2(P_W[1], -P_W[0])

#     Pz, Px, Py = P_W[2]-r2, \
#                  P_W[0] - np.cos(theta1_i) * a1, \
#                  P_W[1]+np.sin(theta1_ii) * a1
#     c3 = (Pz**2 + Px**2 + Py**2 - r3**2 - r4**2) / (2 * r3 * r4)

#     theta3_i = np.arccos(c3)
#     theta3_ii = - theta3_i

#     s3_pos = np.sqrt(1-c3**2)
#     s3_neg = -np.sqrt(1-c3**2)

#     c2_poss3 = (np.sqrt(Px**2 + Py**2) * (r3+r4*c3) +
#             Pz * r4 * s3_pos)/(r3**2+r4**2+2*r3*r4*c3)

#     c2_negs3 = (np.sqrt(Px**2 + Py**2) * (r3+r4*c3) +
#             Pz * r4 * s3_neg)/(r3**2+r4**2+2*r3*r4*c3)

#     theta2_i = np.arccos(c2_poss3)
#     theta2_ii = np.arccos(-c2_poss3)
#     theta2_iii = np.arccos(c2_negs3)
#     theta2_iv = np.arccos(-c2_negs3)

#     alt1 = [theta1_i, theta2_i , theta3_i - np.arctan(35/420)]
#     alt2 = [theta1_i, theta2_iii , theta3_ii - np.arctan(35/420)]
#     alt3 = [theta1_ii, theta2_ii , theta3_i + np.arctan(35/420)]
#     alt4 = [theta1_ii, theta2_iv , theta3_ii + np.arctan(35/420)]

#     liste = np.array([alt1,alt2,alt3,alt4])

#     return liste


    

T_SW = tools.makeT_SW(T_SB)

print('T_SW', T_SW)

P_W = tools.pointFromT(T_SW)

print('P_W', P_W)

theta1_i = np.arctan2(-P_W[1], P_W[0])

theta1_ii = np.arctan2(P_W[1], -P_W[0])

print('theta1: ', theta1_i, theta1_ii)

Pz, Px, Py = P_W[2]-r2, P_W[0] - \
    np.cos(theta1_i) * a1, P_W[1]+np.sin(theta1_ii) * a1

c3 = (Pz**2 + Px**2 + Py**2 - r3**2 - r4**2) / (2 * r3 * r4)

print('c3:', c3)
#c3_ii = ((P_W[2] - (r1+r2))**2 + (P_W[0] - np.cos(theta1_i)*a1)**2 + (P_W[1]+np.sin(theta1_ii) * a1)**2) / (2 * r3 * r4)

theta3_i = np.arccos(c3)
theta3_ii = - theta3_i

s3_pos = np.sqrt(1-c3**2)
s3_neg = -np.sqrt(1-c3**2)

print('theta3 fra c2 og dobbelsjekk fra s3: ', theta3_i, np.arcsin(s3_neg))

print('s3 positiv og negativ:', s3_pos, s3_neg)

# theta3_i = np.arctan2(s3_pos, c3)*Pz
# theta3_ii = -theta3_i

c2_poss3 = (np.sqrt(Px**2 + Py**2) * (r3+r4*c3) +
            Pz * r4 * s3_pos)/(r3**2+r4**2+2*r3*r4*c3)

c2_negs3 = (np.sqrt(Px**2 + Py**2) * (r3+r4*c3) +
            Pz * r4 * s3_neg)/(r3**2+r4**2+2*r3*r4*c3)

theta2_i = np.arccos(c2_poss3)
theta2_ii = np.arccos(-c2_poss3)
theta2_iii = np.arccos(c2_negs3)
theta2_iv = np.arccos(-c2_negs3)

print('theta 2 (i, ii, iii, iv): ', theta2_i, theta2_ii, theta2_iii, theta2_iv)


# dersom s3 er positiv:
# theta2_i = np.arctan2((r3+r4*c3)*Pz - r4*s3_pos*np.sqrt(Px **2 + Py**2), (r3+r4)*np.sqrt(Px**2+Py**2)+r4*s3_pos*Pz)
# theta2_ii = np.arctan2((r3+r4*c3)*Pz + r4*s3_pos*np.sqrt(Px **2 + Py**2), -(r3+r4)*np.sqrt(Px**2+Py**2)+r4*s3_pos*Pz)

# dersom s3 er negativ:
# theta2_iii = np.arctan2((r3+r4*c3)*Pz - r4*s3_neg*np.sqrt(Px **2 + Py**2), (r3+r4)*np.sqrt(Px**2+Py**2)+r4*s3_neg*Pz)
# theta2_iv = np.arctan2((r3+r4*c3)*Pz + r4*s3_neg*np.sqrt(Px **2 + Py**2), -(r3+r4)*np.sqrt(Px**2+Py**2)+r4*s3_neg*Pz)


alt1 = [theta1_i, theta2_i , theta3_i - np.arctan(35/420)]
alt2 = [theta1_i, theta2_iii , theta3_ii - np.arctan(35/420)]
alt3 = [theta1_ii, theta2_ii , theta3_i + np.arctan(35/420)]
alt4 = [theta1_ii, theta2_iv , theta3_ii + np.arctan(35/420)]

print(alt1, '\n', alt2, '\n', alt3, '\n', alt4)


# Alternative kombinasjoner
# 1: (θ1_I,θ2_I,θ3_I)
# 2: (θ1_I,θ2_III,θ3_II)
# 3: (θ1_II,θ2_II,θ3_I)
# 4: (θ1_II,θ2_IV,θ3_II)

print(tools.inverseKinematicsTheta123(T_SB))


T_SW [[0.    0.    1.    0.9  ]
 [0.    1.    0.    0.   ]
 [1.    0.    0.    0.435]
 [0.    0.    0.    1.   ]]
P_W [0.9   0.    0.435]
theta1:  -0.0 3.141592653589793
c3: 0.9965457582448798
theta3 fra c2 og dobbelsjekk fra s3:  0.08314123188843929 -0.0831412318884393
s3 positiv og negativ: 0.08304547985373804 -0.08304547985373804
theta 2 (i, ii, iii, iv):  1.4901161193847656e-08 3.141592638688632 0.07995737424657921 3.0616352793432142
[-0.0, 1.4901161193847656e-08, -1.9290125052862095e-15] 
 [-0.0, 0.07995737424657921, -0.1662824637768805] 
 [3.141592653589793, 3.141592638688632, 0.1662824637768805] 
 [3.141592653589793, 3.0616352793432142, 1.9290125052862095e-15]
[[-0.00000000e+00  1.49011612e-08 -1.92901251e-15]
 [-0.00000000e+00  7.99573742e-02 -1.66282464e-01]
 [ 3.14159265e+00  3.14159264e+00  1.66282464e-01]
 [ 3.14159265e+00  3.06163528e+00  1.92901251e-15]]


In [12]:

config_fasit = np.array([0,-np.pi/4, np.pi/6, 0,0,0])

config1 = np.array([-0.0, 1.0224057635365467, 0.08314123188844062-0.083141231, 0, 0, 0])
config2 = np.array([0, -np.pi/2, np.pi/2, 0, 0, 0])
config3 = np.array([0, 0, np.pi/2, 0, 0, 0])
config4 = np.array([theta1_i, theta2_i , theta3_i - np.arctan(35/420),0,0,0])
config5 = np.array([theta1_i, theta2_iii , theta3_ii - np.arctan(35/420),0,0,0])
config6 = np.array([theta1_ii, theta2_ii , theta3_i + np.arctan(35/420),0,0,0])
config7 = np.array([theta1_ii, theta2_iv , theta3_ii + np.arctan(35/420),0,0,0])




kr6_fasit = KR6(config_fasit) #instans av robot

P_SW_fasit = tools.pointFromT(kr6_fasit.Tsw)

thetalist = kr6_fasit.aIK_thetaLists #liste av forskjellige theta1,2,3 som gir riktig T_SW

config_test = np.array([thetalist[0][0], thetalist[0][1], thetalist[0][2]- np.arctan(35/420), 0,0,0])

kr6_test = KR6(config_test) #instans av robot

P_SW_test = tools.pointFromT(kr6_test.Tsw)

print(P_SW_test)
print(P_SW_fasit)

print(np.allclose(P_SW_fasit, P_SW_test, 0.01, 0.01))

# kr6.add_FK_to_viewer(viewer)

# print(f"Number of iteration: {kr6.IK[3]}")

# kr6.add_nIK_to_viewer(viewer)
viewer = Viewer(background="white")
viewer.add(Grid())
kr6_fasit.add_FK_to_viewer(viewer)


[8.09114166e-01 8.18041462e-17 1.74260142e-02]
[7.43363766e-01 1.08548710e-16 8.64244988e-01]
False


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…